# Sea Ice Thickness Variability Near Kivalina, Alaska (updates through to April 2023)

**Summary**: In this notebook, I am adapting code from Alek Petty, using the updated the monthly gridded winter Arctic sea ice thickness dataset IS2SITMOGR4, which is an ICESat-2 Sea Ice Data product for the circumpolar Arctic from 2018 - 2023. This newly available 2023 data allows us to assess finer resolution changes in sea ice thickness near coastal Arctic commuinities for the first time via satellite data. 

This notebook aims to investigate the following question(s): **1) Has sea ice thickness around critical marine hunting zones in Kivalina, AK changed in recent years?** and, **2) How and where is sea ice thickness changing near Kivalina, AK?**

These outputs will support my master's thesis research, and will aim to validate and support on-the-ground experiences felt by marine subsistence hunters in Kivalina, AK. These visualizations will be presented to hunters in Kivalina and discussions regarding their accuracy and alignment with on-the-ground experiences will be shared. 

**Authors**: Ana Stringer

**Version history**: Version 1 (1/28/2023)
**Edits by Ana Stringer**: Version 2 (3/18/2024) 

### Import Notebook Dependencies

In [1]:
import sys
print(sys.executable)



/Users/anastringer_1/miniconda3/bin/python


In [2]:
# For working with gridded climate data 
import xarray as xr 
# Helper function for reading the data from the bucket
from utils.read_data_utils import read_IS2SITMOGR4 
from utils.plotting_utils import static_winter_comparison_lineplot, staticArcticMaps, staticArcticMaps_overlayDrifts, interactiveArcticMaps, compute_gridcell_winter_means, interactive_winter_comparison_lineplot # Plotting utils 
import numpy as np
# Plotting dependencies
#%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
# Sets figure size in the notebook
mpl.rcParams['figure.dpi'] = 150 

# Remove warnings to improve display
import warnings 
warnings.filterwarnings('ignore') 



ModuleNotFoundError: No module named 'xarray'

In [ ]:
#!conda install zarr
!conda install ipywidgets 

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/anastringer_1/miniconda3

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hf0a4a13_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    conda-24.1.2               |  py311h267d04e_0         1.2 MB  conda-forge
    ipywidgets-8.1.2           |     pyhd8ed1ab_0         111 KB  conda-forge
    jupyterlab_widgets-3.0.10  |     pyhd8ed1ab_0         183 KB  conda-forge
    widgetsnbextension-4.0.10  |     pyhd8ed1ab_0         866 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following packages will be UPDATED:

  ca-certif

### Read in the Version 3 monthly gridded winter Arctic sea ice data
As before, there are multiple ways of accessing this dataset. The 'official' method is to download the latest data direct from the NSIDC at the following link: https://nsidc.org/data/is2sitmogr4/. 

However, to simplify access and avoid the need to even go through the NSIDC I have uploaded the raw netcdfs and an aggregated zarr dataset to an AWS S3 bucket. The benefit of the zarr approach is that you can read the data in on the fly which negates the need to store a local copy of the data. Performance is improved when doing this analysis on the same AWS platform. 

The new Version 3 (V3) data has been re-processed since the start of the mission (November 2018 to April 2023 currently available) so I fully recommend downloading the entire dataset afresh. Note that in V3 the xgrid/ygrid variables got renamed to x/y. New variables have also been included in V3 which are discussed below

In [ ]:
IS2SITMOGR4_v3 = read_IS2SITMOGR4(data_type='zarr-s3', 
                                   local_data_path='./data/IS2SITMOGR4/', version='V3', download=False,
                                   persist=True) 

IS2SITMOGR4_v3

In [ ]:
IS2SITMOGR4_v3 = read_IS2SITMOGR4(data_type='zarr-s3', 
                                   local_data_path='./data/IS2SITMOGR4/', version='V3', download=False,
                                   persist=True) 

IS2SITMOGR4_v3

making a quick plot to make sure that data read in correctly 

In [ ]:
# Years over which to perform analysis (start year of that winter period)
years = [x for x in range(2018, 2022+1)]

thickness_winter_means = compute_gridcell_winter_means(IS2SITMOGR4_v3.ice_thickness_int, years=years)
staticArcticMaps(thickness_winter_means, dates=thickness_winter_means.time.values,title="", set_cbarlabel = "Sea ice thickness (m)", col_wrap=3, cmap="viridis", vmin=0, vmax=5, out_str='thickness_winter_2018_2023')

In [ ]:
# Define the map extent around Kivalina, AK (longitude and latitude bounds)
kivalina_extent = [-165.5, -163.5, 67.2, 68.2]

# Use the staticArcticMaps function to create the plot with the specified Kivalina extent
fig = staticArcticMaps(thickness_winter_means, 
                       title="Zoomed-in view of Kivalina", 
                       dates=thickness_winter_means.time.values, 
                       out_str='thickness_winter_2018_2023', 
                       cmap="viridis", 
                       vmin=0, 
                       vmax=5, 
                       col_wrap=3, 
                       set_cbarlabel="Sea ice thickness (m)", 
                       savefig=True, 
                       zoom_extent=kivalina_extent)

# Display the figure if you are in a Jupyter notebook environment; otherwise, it saves the figure to a file.
# If using a Jupyter notebook, uncomment the line below:
plt.show(fig)


In [ ]:
# Read in the raw monthly gridded winter Arctic sea ice data from S3
IS2SITMOGR4_all = read_IS2SITMOGR4() 
IS2SITMOGR4_all

### Defining a Region of Interest around Kivalina, AK

In [ ]:
# Print out the variables and coordinates in the dataset
print(IS2SITMOGR4_all)

# Check if the data contains entries for the year 2023
print("Data variables available:", IS2SITMOGR4_all.data_vars)
print("Coordinates available:", IS2SITMOGR4_all.coords)

# Specifically check for the 'ice_thickness_int' variable and the time period of interest
if 'ice_thickness_int' in IS2SITMOGR4_all.data_vars:
    print("Ice thickness variable found.")
    if 2023 in IS2SITMOGR4_all['time'].dt.year:
        print("Data for the year 2023 is available.")
    else:
        print("Data for the year 2023 is not available.")
else:
    print("Ice thickness variable not found. Available variables:", IS2SITMOGR4_all.data_vars)


In [ ]:
import utils.plotting_utils as utils
thickness_winter_means = utils.compute_gridcell_winter_means(IS2SITMOGR4_v3.ice_thickness_int, years=years)


In [ ]:
# #checks 
# # Inside staticArcticMaps function, where the plotting occurs:
# for ax in ax_iter:
#     # Other plotting code...
#     if custom_extent:  # Make sure this condition checks if all custom_extent values are not None
#         ax.set_extent(custom_extent, crs=ccrs.PlateCarree())
#     # Other plotting code...


In [ ]:
def add_scale_bar(ax, scale_len, location=(0.5, 0.05), linewidth=3):
    """
    Add a scale bar to the given axes.
    
    Args:
    ax: Matplotlib axis object to draw the scale bar on.
    scale_len: Length of the scale bar in miles.
    location: Tuple of (x, y) to place the scale bar on the axes. Values are relative to the axes size.
    linewidth: Thickness of the scale bar line.
    """
    # Transform length in miles to degrees latitude (roughly)
    length_degrees = scale_len / 69.0  # approximate conversion from miles to degrees latitude
    
    # Determine scale bar coordinates in axes coordinates
    x, y = location
    bar_xs = [x - length_degrees / 2, x + length_degrees / 2]
    bar_ys = [y, y]
    
    # Transform axes coordinates to data coordinates
    inverse = ccrs.PlateCarree()._as_mpl_transform(ax).inverted()
    bar_x_data, bar_y_data = inverse.transform(np.vstack((bar_xs, bar_ys)).T)
    
    # Draw the scale bar
    ax.plot(bar_x_data, bar_y_data, transform=ccrs.PlateCarree(), color='black', linewidth=linewidth)
    
    # Label the scale bar
    ax.text(x, y - 0.1, f'{scale_len} miles', transform=ccrs.PlateCarree(), horizontalalignment='center', verticalalignment='bottom')


### check to see if there's any data for 11-2022

In [ ]:
# Check if there is data for November 2022
nov_2022_data = IS2SITMOGR4_all.sel(time='2022-11').ice_thickness_int

# Print the summary of the data for November 2022
print(nov_2022_data)

# If the data exists, check for any masking or missing values
print("Number of non-NaN values:", np.count_nonzero(~np.isnan(nov_2022_data)))

# Check the range of values for this month
print("Data range for November 2022:", np.nanmin(nov_2022_data.values), "to", np.nanmax(nov_2022_data.values))

# Adjust the colormap range to include the minimum and maximum data values
vmin = 0.0
vmax = 7.0

# Plot the data for November 2022
nov_2022_data.plot(
    vmin=vmin, vmax=vmax, cmap='viridis',
    subplot_kws={'projection': ccrs.NorthPolarStereo(central_longitude=-155)}, 
    transform=ccrs.PlateCarree()
)
plt.show()

